<a href="https://colab.research.google.com/github/esn73/lab_python/blob/master/ml03_train_test_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 머신 러닝의 과정
    * 문제 정의 --> 데이터 준비 --> 데이터 탐색 --> 훈련 셋, 검증 셋, 테스트 셋 분리 --> 모델 선택, 훈련 --> 모델 평가 --> 모델 튜닝
* 훈련 셋, 검증 셋, 테스트 셋으로 나누는 방법
    * 순차 추출(sequential sampling)
    * 임의 추출(random sampling)
    * 층화 추출(stratified sampling)
* 분류 모델 평가 지표
    * 정확도(accuracy)
    * 정밀도(precision)
    * 재현율(recall)
    * F1-score
    * 민감도(sensitivity)
    * 특이도(spei
* 특성 스케일링
    * 표준화(standardization)
    * 정규화(normalization)

# 필요한 모듈, 클래스, 함수들을 import

In [30]:
import numpy as np # ndarray(배열) 함수, 메서드
import pandas as pd # Series, DataFrame 함수, 메서드
import matplotlib.pyplot as plt # 시각화
import seaborn as sns # 시각화

# Machine Learning
# 머신 러닝 모델(알고리즘) - KNN 분류기
from sklearn.neighbors import KNeighborsClassifier
# 훈련/테스트 셋 분리 함수
from sklearn.model_selection import train_test_split
# 모델 평가 지표 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 특성 스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 준비

* fish.csv 파일: 어종, 특성들(무게, 길이, ...)
* 2진 분류(도미, 빙어)
* 특성 

In [3]:
fish_scv = 'https://github.com/JakeOh/202110_itw_lab_python/raw/main/fish.csv'

In [4]:
fish = pd.read_csv(fish_scv)

In [5]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [6]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [8]:
df = fish.loc[fish['Species'].isin(['Bream', 'Smelt']),
              ['Species', 'Weight', 'Length']]
df

,Species,Weight,Length
0,Bream,242.0,25.4
1,Bream,290.0,26.3
2,Bream,340.0,26.5
3,Bream,363.0,29.0
4,Bream,430.0,29.0
5,Bream,450.0,29.7
6,Bream,500.0,29.7
7,Bream,390.0,30.0
8,Bream,450.0,30.0
9,Bream,500.0,30.7


In [9]:
df.head()

,Species,Weight,Length
0,Bream,242.0,25.4
1,Bream,290.0,26.3
2,Bream,340.0,26.5
3,Bream,363.0,29.0
4,Bream,430.0,29.0


In [12]:
# DataFrame을 특성과 타겟으로 분리하고 ndarray(배열)로 변환
features = df.iloc[:, 1:].values #DataFrame이기때문에 values를 사용해서 배열로 변환
target = df['Species'].values

In [14]:
features.shape # (n_samples, n_features)

(49, 2)

In [15]:
target.shape # (n_samples,)

(49,)

In [16]:
features[:5, :]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [17]:
target[:5]

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream'], dtype=object)

In [19]:
np.unique(target, return_counts=True) # 유일한 값과 개수

(array(['Bream', 'Smelt'], dtype=object), array([35, 14]))

## 훈련 셋, 테스트 셋 분리

## 순차 추출(sequential sampling)

* 훈련 셋:테스트 셋 = 7:3 
    * (거의 사용하지 X)

In [21]:
49 * 0.7, 49 * 0.3

(34.3, 14.7)

In [22]:
num_test = 35   # 훈련 셋의 샘플 개수 -> 테스트 셋의 샘플 개수: 48-35 = 14

In [23]:
X_train = features[:num_test, :] # 훈련 셋
y_train = target[:num_test]      # 훈련 레이블
X_test = features[num_test:, :]  # 테스트 셋
y_test = target[num_test:]       # 테스트 레이블

In [24]:
np.unique(y_train)
#> 훈련 셋에는 물고기의 종류가 모두 도미(Bream)만 있음.

array(['Bream'], dtype=object)

In [25]:
np.unique(y_test)
#> 테스트 셋에는 모두 빙어(Smelt)만 있음.

array(['Smelt'], dtype=object)

In [32]:
# train_test_split 함수를 사용한 순차 추출
X_train, X_test, y_train, y_test = train_test_split(features, target, # train_size or test_size중 하나만 기재하면 나머지가 계산되서 들어감,
                                                    train_size=35, # 훈련 셋의 샘플 개수
                                                    shuffle=False)  # 샘플을 섞지 않음 -> 순차 추출

In [33]:
X_train.shape, X_test.shape

((35, 2), (14, 2))

In [34]:
y_train.shape, y_test.shape

((35,), (14,))

In [35]:
np.unique(y_train)

array(['Bream'], dtype=object)

In [36]:
np.unique(y_test)

array(['Smelt'], dtype=object)